In [9]:
import json
import random

In [10]:
# Create single attack in dictionary format
def generate_single_attack_dict(name:str, 
                                attack_type:str, 
                                start:int, 
                                end:int, 
                                target:str, 
                                tags:list=None
                                ):
    attack_dict = {
        'name': name,
        'type': attack_type,
        'trigger': {
            'type': 'time',
            'start': start,
            'end': end
        },
        'tags': tags,
        'target': target
    }
    
    return attack_dict

In [11]:
# Choice of tag to involve in the selected attack based on attack_type and target 
def draw_attacked_tag_list(attack_type, target, tags_config):
    tags = []
    if attack_type == 'DOS':
        return None
    
    elif attack_type == 'MITM':
        # Get only one tag (modify to ask for multiple tags)
        n_sampels = 1
        sampled_tags = random.sample(tags_config[target], n_sampels)
        
        for tag in sampled_tags:
            var_name = tag['tag']
            prop = random.choice(tag['property'])
            value = random.choice(tag['value'])
        
            final_tag = {'tag': var_name, 'property': prop, 'value': value}
            tags.append(final_tag)
        
        return tags
        
    else:
        raise Exception("Wrong attack type!")

In [12]:
# Build weekly attacks 
def build_weekly_attacks_list(attack_types:list, 
                              prob_types:list,
                              targets:list, 
                              tags_config:dict, 
                              presence_chance:float, 
                              max_occurrences:int, 
                              episode_length:int, 
                              durations:list, 
                              double_attack_chance:float
                              ) -> list:
    weekly_attacks = []
    
    presence_probability = random.uniform(0, 1)
    
    if presence_probability < presence_chance:
        n_attacks = random.randint(1, max_occurrences)

        last_timestep = 0
        for i in range(n_attacks):
            name = 'attack_' + str(i)
            
            # Sample attack type (unfair coin flip)
            attack_type = random.choices(attack_types, weights=prob_types, k=1)[0]
            
            # Sample attack trigger
            duration = random.choice(durations)
            ith_lower_bound = (episode_length // n_attacks) * i
            lower_bound = ith_lower_bound if ith_lower_bound * i < last_timestep else last_timestep
            upper_bound = (episode_length // n_attacks) * i + (episode_length // n_attacks)
            start_time = random.randint(lower_bound, upper_bound - duration - 1)
            end_time = start_time + duration
            last_timestep = end_time
            
            # Double attack on both PLCs
            if random.uniform(0, 1) < double_attack_chance:
                for i, target in enumerate(targets):
                    tags = draw_attacked_tag_list(attack_type, target, tags_config)
                    # Append the attack to the list of daily attacks
                    weekly_attacks.append(generate_single_attack_dict(name=name + '_{}'.format(i+1),
                                                                      attack_type=attack_type,
                                                                      start=start_time,
                                                                      end=end_time,
                                                                      target=target,
                                                                      tags=tags))
            # Single attack
            else:
                # Sample attack target
                target = random.choice(targets)
                # Sample attacked tags based on type and target
                tags = draw_attacked_tag_list(attack_type, target, tags_config)
                # Append the attack to the list of daily attacks
                weekly_attacks.append(generate_single_attack_dict(name=name,
                                                                  attack_type=attack_type,
                                                                  start=start_time,
                                                                  end=end_time,
                                                                  target=target,
                                                                  tags=tags))
    return weekly_attacks
        

In [34]:
# Generate all the required yaml files, storing weekly attacks and
def generate_weeks_under_attack(n_weeks:int, where_to_store:str, **kwargs):
    dataset = []
    
    for _ in range(n_weeks):
        attacks_list = build_weekly_attacks_list(attack_types=kwargs['attack_types'], 
                                                 prob_types=kwargs['prob_types'],
                                                 targets=kwargs['targets'], 
                                                 tags_config=kwargs['tags_config'], 
                                                 presence_chance=kwargs['presence_chance'], 
                                                 max_occurrences=kwargs['max_occurrences'],
                                                 episode_length=kwargs['episode_length'],
                                                 durations=kwargs['durations'],
                                                 double_attack_chance=kwargs['double_attack_chance'])
        if attacks_list:
            #yaml_path = where_to_store + 'rand_attacks_' + str(week) + '.yaml'
            #with open(yaml_path, 'w') as fout:
            #    yaml.dump(attacks_list, fout)
            dataset.append(attacks_list)
        
        else:
            dataset.append([])
    
    with open(where_to_store, 'w') as fout:
        json.dump(dataset, fout)

In [35]:
params = {}
params['attack_types'] = ['MITM', 'DOS']
params['prob_types'] = [0.7, 0.3]
params['targets'] = ['PLC2', 'PLC3']
params['tags_config'] = {'PLC2': [{'tag': 'T41', 'property': ['pressure'], 'value': [0, 10.6]}],
                        'PLC3': [{'tag': 'T42', 'property': ['pressure'], 'value': [0, 10.6]}]}
params['max_occurrences'] = 5
params['durations'] = [28800, 43200, 57600, 86400]
params['episode_length'] = 604800   # 7 days
params['presence_chance'] = 0.7
params['double_attack_chance'] = 0.05

n_weeks = 5000
where_to_store = 'attacks_sensors_5000_prob0_7.json'

generate_weeks_under_attack(n_weeks=n_weeks, where_to_store=where_to_store, **params) 

In [36]:
# Read the json file saved
with open(where_to_store, 'r') as f:
    data = json.load(f)

In [38]:
data[0]

[{'name': 'attack_0',
  'type': 'MITM',
  'trigger': {'type': 'time', 'start': 25230, 'end': 82830},
  'tags': [{'tag': 'T41', 'property': 'pressure', 'value': 10.6}],
  'target': 'PLC2'},
 {'name': 'attack_1',
  'type': 'MITM',
  'trigger': {'type': 'time', 'start': 204633, 'end': 291033},
  'tags': [{'tag': 'T42', 'property': 'pressure', 'value': 10.6}],
  'target': 'PLC3'},
 {'name': 'attack_2',
  'type': 'MITM',
  'trigger': {'type': 'time', 'start': 366055, 'end': 394855},
  'tags': [{'tag': 'T42', 'property': 'pressure', 'value': 0}],
  'target': 'PLC3'}]